In [8]:
import cv2
import numpy as np
from ultralytics import YOLO
from calculates import Calculates
from constants import Constants as K
import time

In [6]:
def valid_keypoints(confs, indices, threshold=0.3):
    """Retorna True si todos los keypoints de los índices indicados superan el umbral."""
    return np.all(confs[indices] > threshold)

In [7]:
Calculate = Calculates()
model = YOLO('yolo11n-pose.pt')

In [ ]:
exercise = 'pushup'
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)
    annotated_frame = results[0].plot()

    if results[0].keypoints is not None and len(results[0].keypoints.xy) > 0:
        if exercise == "curl":
            relevant_indices = [K.YOLO_POSE_KEYPOINTS['RIGHT_SHOULDER'],
                                K.YOLO_POSE_KEYPOINTS['RIGHT_ELBOW'],
                                K.YOLO_POSE_KEYPOINTS['RIGHT_WRIST']]
        elif exercise == "squat":
            relevant_indices = [K.YOLO_POSE_KEYPOINTS['RIGHT_HIP'],
                                K.YOLO_POSE_KEYPOINTS['RIGHT_KNEE'],
                                K.YOLO_POSE_KEYPOINTS['RIGHT_ANKLE']]
        elif exercise == "pushup":
            relevant_indices = [K.YOLO_POSE_KEYPOINTS['RIGHT_SHOULDER'],
                                K.YOLO_POSE_KEYPOINTS['RIGHT_ELBOW'],
                                K.YOLO_POSE_KEYPOINTS['RIGHT_WRIST']]
        elif exercise == "plank":
            relevant_indices = [K.YOLO_POSE_KEYPOINTS['RIGHT_SHOULDER'],
                                K.YOLO_POSE_KEYPOINTS['RIGHT_HIP'],
                                K.YOLO_POSE_KEYPOINTS['RIGHT_KNEE']]
        else:
            relevant_indices = []

        # Seleccionar la detección con mayor promedio de confianza en los keypoints relevantes
        best_index = 0
        best_conf = -1
        for i in range(len(results[0].keypoints.xy)):
            confs = results[0].keypoints.conf[i].cpu().numpy()
            mean_conf = np.mean(confs[relevant_indices])
            if mean_conf > best_conf:
                best_conf = mean_conf
                best_index = i

        kpts = results[0].keypoints.xy[best_index].cpu().numpy()
        confs = results[0].keypoints.conf[best_index].cpu().numpy()

        try:
            if exercise == "curl":
                shoulder_idx = K.YOLO_POSE_KEYPOINTS['RIGHT_SHOULDER']
                elbow_idx = K.YOLO_POSE_KEYPOINTS['RIGHT_ELBOW']
                wrist_idx = K.YOLO_POSE_KEYPOINTS['RIGHT_WRIST']

                if valid_keypoints(confs, [shoulder_idx, elbow_idx, wrist_idx]):
                    shoulder = kpts[shoulder_idx]
                    elbow = kpts[elbow_idx]
                    wrist = kpts[wrist_idx]
                    angle = Calculate.angles(shoulder, elbow, wrist)
                    text_pos = tuple(map(int, elbow))
                    cv2.putText(annotated_frame, f"{angle:.1f}", text_pos,
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 5, cv2.LINE_AA)
                    cv2.putText(annotated_frame, f"{angle:.1f}", text_pos,
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
                    if angle > K.CURL_MAX_ANGLE:
                        K.CURL_STAGE = "down"
                    elif angle < K.CURL_MIN_ANGLE and K.CURL_STAGE == 'down':
                        K.CURL_STAGE = "up"
                        K.CURL_COUNTER += 1

            elif exercise == "squat":
                r_shoulder_idx = K.YOLO_POSE_KEYPOINTS['RIGHT_SHOULDER']
                r_hip_idx = K.YOLO_POSE_KEYPOINTS['RIGHT_HIP']
                r_knee_idx = K.YOLO_POSE_KEYPOINTS['RIGHT_KNEE']
                r_ankle_idx = K.YOLO_POSE_KEYPOINTS['RIGHT_ANKLE']

                if valid_keypoints(confs, [r_hip_idx, r_knee_idx, r_ankle_idx]):
                    r_shoulder = kpts[r_shoulder_idx]
                    r_hip = kpts[r_hip_idx]
                    r_knee = kpts[r_knee_idx]
                    r_ankle = kpts[r_ankle_idx]
                    leg_angle = Calculate.angles(r_hip, r_knee, r_ankle)
                    torso_angle = Calculate.angles(r_shoulder, r_hip, r_knee)
                    text_pos_leg = tuple(map(int, r_knee))
                    text_pos_torso = tuple(map(int, r_hip))
                    cv2.putText(annotated_frame, f"Leg: {leg_angle:.1f}", text_pos_leg,
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 5, cv2.LINE_AA)
                    cv2.putText(annotated_frame, f"Leg: {leg_angle:.1f}", text_pos_leg,
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
                    cv2.putText(annotated_frame, f"Torso: {torso_angle:.1f}", text_pos_torso,
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 5, cv2.LINE_AA)
                    cv2.putText(annotated_frame, f"Torso: {torso_angle:.1f}", text_pos_torso,
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
                    if leg_angle < K.SQUAT_MIN_ANGLE and torso_angle < K.SQUAT_TORSO_MIN_ANGLE and K.SQUAT_STAGE != "down":
                        K.SQUAT_STAGE = "down"
                    elif leg_angle > K.SQUAT_MAX_ANGLE and K.SQUAT_STAGE == "down":
                        K.SQUAT_STAGE = "up"
                        K.SQUAT_COUNTER += 1
            elif exercise == "pushup":
                # Se utilizan los mismos keypoints que en curl
                shoulder_idx = K.YOLO_POSE_KEYPOINTS['RIGHT_SHOULDER']
                elbow_idx = K.YOLO_POSE_KEYPOINTS['RIGHT_ELBOW']
                wrist_idx = K.YOLO_POSE_KEYPOINTS['RIGHT_WRIST']

                if valid_keypoints(confs, [shoulder_idx, elbow_idx, wrist_idx]):
                    shoulder = kpts[shoulder_idx]
                    elbow = kpts[elbow_idx]
                    wrist = kpts[wrist_idx]
                    angle = Calculate.angles(shoulder, elbow, wrist)
                    text_pos = tuple(map(int, elbow))
                    cv2.putText(annotated_frame, f"{angle:.1f}", text_pos,
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 5, cv2.LINE_AA)
                    cv2.putText(annotated_frame, f"{angle:.1f}", text_pos,
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
                    if angle < K.PUSHUP_MIN_ANGLE:
                        K.PUSHUP_STAGE = "down"
                    elif angle > K.PUSHUP_MAX_ANGLE and K.PUSHUP_STAGE == "down":
                        K.PUSHUP_STAGE = "up"
                        K.PUSHUP_COUNTER += 1

            elif exercise == "plank":
                # Se usan los keypoints del hombro, cadera y rodilla para evaluar la alineación del torso
                shoulder_idx = K.YOLO_POSE_KEYPOINTS['RIGHT_SHOULDER']
                hip_idx = K.YOLO_POSE_KEYPOINTS['RIGHT_HIP']
                knee_idx = K.YOLO_POSE_KEYPOINTS['RIGHT_KNEE']

                if valid_keypoints(confs, [shoulder_idx, hip_idx, knee_idx]):
                    shoulder = kpts[shoulder_idx]
                    hip = kpts[hip_idx]
                    knee = kpts[knee_idx]
                    torso_angle = Calculate.angles(shoulder, hip, knee)
                    text_pos = tuple(map(int, hip))
                    cv2.putText(annotated_frame, f"Torso: {torso_angle:.1f}", text_pos,
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 5, cv2.LINE_AA)
                    cv2.putText(annotated_frame, f"Torso: {torso_angle:.1f}", text_pos,
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
                    # Si el ángulo es mayor o igual al umbral, la postura es correcta
                    if torso_angle >= K.PLANK_THRESHOLD_ANGLE:
                        if K.PLANK_START_TIME is None:
                            K.PLANK_START_TIME = time.time()
                        elapsed_time = time.time() - K.PLANK_START_TIME
                    else:
                        # Si se pierde la postura, se reinicia el contador
                        K.PLANK_START_TIME = None
                        elapsed_time = 0
                    cv2.putText(annotated_frame, f"Time: {elapsed_time:.1f}s", (10, 100),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        except Exception as e:
            print(f"Error processing keypoints: {e}")

    cv2.rectangle(annotated_frame, (0, 0), (300, 73), (245, 117, 16), -1)
    if exercise == "curl":
        reps = K.CURL_COUNTER
        stage = K.CURL_STAGE
    elif exercise == "squat":
        reps = K.SQUAT_COUNTER
        stage = K.SQUAT_STAGE
    elif exercise == "pushup":
        reps = K.PUSHUP_COUNTER
        stage = K.PUSHUP_STAGE
    elif exercise == "plank":
        reps = "" 
        stage = ""
    else:
        reps = ""
        stage = ""

    cv2.putText(annotated_frame, 'REPS', (15, 12),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(annotated_frame, str(reps),
                (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
    cv2.putText(annotated_frame, 'STAGE', (135, 12),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(annotated_frame, stage if stage is not None else "",
                (90, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

    cv2.imshow('Virtual GYM', annotated_frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()